# Exercicio 01 - Coleta, Preparação e Análise de Dados

### Grupo: Giovani Cancherini, Eduardo Traunig, Vinicius Quintian, João Pedro Fossa
### Data de Entrega: 07/04/2024 (Turma 10)

---

## Tarefa 1 – Desenvolvendo um crawler

Escreva um crawler para descobrir e coletar páginas da Wikipedia em Português. Seu programa deve coletar 5.000 páginas de verbetes diferentes da Wikipedia, à partir da página
inicial: https://pt.wikipedia.org. Não coletar páginas de outros sites e também não deve
ser coletado páginas internas da Wikipedia que não são verbetes.

O crawler deve funcionar da seguinte maneira:
1. Obter uma página.
2. Salvar a página como um arquivo html, chamado <titulo_verbete>.html
3. Extrair todos os links que se encontram nessa página
4. Filtrar os links, removendo os que não se referem à verbetes e os verbetes que já foram
visitados.
1
5. Guardar esses links em uma lista
6. Escolher um link não visitado para ser a próxima página.
7. Voltar ao passo inicial
As páginas de verbetes coletadas deverão ser salvas como arquivos com a extensão .html.
Lembre-se de tomar cuidado para não estressar o servidor com requisições em excesso

In [7]:
import requests

from bs4 import BeautifulSoup

In [8]:
def salvar_links(soup):
    links_obtidos = []
    todos_links = soup.find(id="bodyContent").find_all("a")
    
    for link in todos_links:
        if "href" in link.attrs.keys() and link["href"].startswith("/wiki/"):
            if link["href"].find(":") == -1:
                links_obtidos.append(link["href"])
                # print(links_obtidos[-1])
                
    return links_obtidos

In [9]:
import os

def salvar_html(nome_arquivo, response):
    subpath = "Exercicio1-paginas/"
    
    os.makedirs(subpath, exist_ok=True)
    
    fullpath = os.path.join(subpath, f"{nome_arquivo}.html")
    
    with open(fullpath, "w", encoding="utf-8") as f:
        conteudo = response.content.decode("utf-8")
        f.write(conteudo)

In [10]:
import re

def sanitize_filename(filename):
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

In [11]:
import time 

lista_paginas_visitadas = []
lista_paginas_nao_visitadas = []
url_base = "https://pt.wikipedia.org"

def obtem_pagina_recursivamente(url, contador_repeticoes):
    if contador_repeticoes == 0:
        print()
        print()
        print("[SUCESSO]")
        print("Todas as paginas obtidas!")
        return
    
    time.sleep(1)
    
    response = requests.get(url)
    lista_paginas_visitadas.append(url)
    
    soup = BeautifulSoup(response.content, "html.parser")
    titulo = soup.select(".mw-page-title-main")
    nome_arquivo = titulo[0].text.strip().replace(" ", "_") if titulo else "pagina_sem_titulo"
    nome_arquivo = sanitize_filename(nome_arquivo)  # Limpa caracteres inválidos
    
    links_obtidos = salvar_links(soup)
    salvar_html(nome_arquivo, response)
    
    for link in links_obtidos:
        link_completo = url_base + link
        if link_completo not in lista_paginas_visitadas and link_completo not in lista_paginas_nao_visitadas:
            lista_paginas_nao_visitadas.append(link_completo)
       
    if lista_paginas_nao_visitadas:
        lista_paginas_nao_visitadas.pop(0)
    else:
        print()
        print()
        print("[ATENCAO]")
        print("Nao ha mais paginas para visitar!")
        
        
    if lista_paginas_nao_visitadas:
        return obtem_pagina_recursivamente(lista_paginas_nao_visitadas[0], contador_repeticoes - 1)
    else:
        print()
        print()
        print("[ATENCAO]")
        print("Nao ha mais paginas para visitar!")
        return

In [12]:
def obtem_pagina_iterativo(url, contador_repeticoes):
    lista_paginas_visitadas = set()
    lista_paginas_nao_visitadas = [url]
    url_base = "https://pt.wikipedia.org"

    while lista_paginas_nao_visitadas and contador_repeticoes > 0:
        url_atual = lista_paginas_nao_visitadas.pop(0)
        if url_atual in lista_paginas_visitadas:
            continue
        
        print(f"[INFO] Visitando: {url_atual}")
        time.sleep(1)

        response = requests.get(url_atual)
        lista_paginas_visitadas.add(url_atual)

        soup = BeautifulSoup(response.content, "html.parser")
        titulo = soup.select(".mw-page-title-main")
        nome_arquivo = titulo[0].text.strip().replace(" ", "_") if titulo else "pagina_sem_titulo"
        nome_arquivo = sanitize_filename(nome_arquivo)
        
        salvar_html(nome_arquivo, response)
        links_obtidos = salvar_links(soup)

        for link in links_obtidos:
            link_completo = url_base + link
            if link_completo not in lista_paginas_visitadas and link_completo not in lista_paginas_nao_visitadas:
                lista_paginas_nao_visitadas.append(link_completo)
        
        contador_repeticoes -= 1

    print("\n[SUCESSO] Todas as páginas obtidas!")

In [13]:
# Inicia a busca
## Verificar output em /Exercicio1-paginas/

pagina_de_inicio = "https://pt.wikipedia.org"
numero_de_paginas = 5000 # para teste final = 5000

# obtem_pagina_recursivamente(pagina_de_inicio, numero_de_paginas)
obtem_pagina_iterativo(pagina_de_inicio, numero_de_paginas)

[INFO] Visitando: https://pt.wikipedia.org
[INFO] Visitando: https://pt.wikipedia.org/wiki/Wikip%C3%A9dia
[INFO] Visitando: https://pt.wikipedia.org/wiki/Enciclop%C3%A9dia
[INFO] Visitando: https://pt.wikipedia.org/wiki/Conte%C3%BAdo_livre
[INFO] Visitando: https://pt.wikipedia.org/wiki/L%C3%ADngua_portuguesa
[INFO] Visitando: https://pt.wikipedia.org/wiki/Jason_Sendwe
[INFO] Visitando: https://pt.wikipedia.org/wiki/Pol%C3%ADtico
[INFO] Visitando: https://pt.wikipedia.org/wiki/Rep%C3%BAblica_Democr%C3%A1tica_do_Congo
[INFO] Visitando: https://pt.wikipedia.org/wiki/Rep%C3%BAblica_do_Congo_(L%C3%A9opoldville)
[INFO] Visitando: https://pt.wikipedia.org/wiki/Tanganhica_(Rep%C3%BAblica_Democr%C3%A1tica_do_Congo)
[INFO] Visitando: https://pt.wikipedia.org/wiki/Congo_Belga
[INFO] Visitando: https://pt.wikipedia.org/wiki/Lubas
[INFO] Visitando: https://pt.wikipedia.org/wiki/Metodismo
[INFO] Visitando: https://pt.wikipedia.org/wiki/Enfermagem
[INFO] Visitando: https://pt.wikipedia.org/wiki/Prot

---

## Tarefa 2 – Extraindo informações de Infoboxes

A segunda tarefa consiste em identificar as páginas que possuem infoboxes, que são usadas
para resumir as informações de um artigo na Wikipédia. Veja na figura a seguir um exemplo
de página que contém um infobox (ele está destacado em vermelho na imagem).
Infoboxes estruturam informações de diversas maneiras, por isso é difícil conseguir extrair
todos os seus elementos de forma fácil. Portanto, focaremos nossos esforços em extrair apenas
alguns elementos. São eles:
1. Título: toda infobox possui um título que fica no topo da caixa. No exemplo da figura,
o título é Alan Turing
2. Pares chave – valor: esses pares são identificados por uma chave que está associada a
um único valor. Por exemplo, na figura temos a chave “Nome completo” e o valor “Alan
Mathison Turing”.
3. Pares chave – lista: nesse tipo de item, uma chave está associada a uma lista de valores.
Na figura de exemplo são pares de chave – lista os campos: Conhecido(a) por, Alma
mater, Orientado(a)(s), Instituições e Campos.
Sua tarefa consiste em extrair o conteúdo das infoboxes de todas as páginas que foram extraídas no exercício anterior. Seu programa deve identificar quando uma página possui uma
infobox, realizar a extração das informações e salvá-las em um arquivo .json cujo nome é o
título da infobox.
Para fins de teste, será fornecido um conjunto de páginas juntamente com a saída esperada
para cada uma delas.

Sua tarefa consiste em extrair o conteúdo das infoboxes de todas as páginas que foram extraídas no exercício anterior. Seu programa deve identificar quando uma página possui uma
infobox, realizar a extração das informações e salvá-las em um arquivo .json cujo nome é o
título da infobox.
Para fins de teste, será fornecido um conjunto de páginas juntamente com a saída esperada
para cada uma delas.


In [19]:
import os
import json
from bs4 import BeautifulSoup

subpath = "Exercicio1-paginas/"
output_path = "Exercicio1-jsons/"

os.makedirs(output_path, exist_ok=True)

def extrair_infobox(soup):
    box = soup.find("table", class_="infobox infobox_v2")

    if not box:
        return None
    info = {}

    # 1. Extrair o titulo da infobox
    titulo_tag = box.find("th", class_="topo padrao")
    if titulo_tag:
        titulo = titulo_tag.get_text(strip=True)
        info["Título"] = titulo if isinstance(titulo, str) else "Título_Desconhecido"
    else:
        return None

    # 2. Extrair os pares chave-valor e chave-lista
    for tag in box.find_all("tr"):
        # Procurar chave dentro de <th> ou <td scope="row">
        chave_tag = tag.find("th") or tag.find(attrs={"scope": "row"})
        if chave_tag:
            chave = chave_tag.get_text(strip=True)
            valores = tag.find_all("td")  # Valores associados

            # Se houver apenas um <td>, e um valor unico; senao, e uma lista
            if len(valores) == 1:
                valor = valores[0].get_text(" | ", strip=True)
            else:
                valor = [td.get_text(strip=True) for td in valores]

            info[chave] = valor

    return info


In [23]:
def salvar_json(file):
    fullpath = os.path.join(subpath, file)
    with open(fullpath, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    info = extrair_infobox(soup)

    if info:
        if isinstance(info["Título"], list):
            info["Título"] = " ".join(info["Título"])  # Junta elementos da lista em uma única string

        filename = f"{info['Título'].replace(' ', '_')}.json"
        filename = sanitize_filename(filename)
        
        json_path = os.path.join(output_path, filename)

        # Salvar os dados extraídos
        with open(json_path, "w", encoding="utf-8") as json_file:
            json.dump(info, json_file, indent=4, ensure_ascii=False)

        print(f"[SUCESSO] | Informacoes extraidas e salvas                                  => {filename}")
    else:
        print(f"[ATENCAO] | {file} | Nao contem infobox")

In [21]:
def processar_todas_as_paginas():
    arquivos = os.listdir(subpath)
    
    for file in arquivos:
        salvar_json(file)

In [24]:
# Inicia a busca
## Verificar output em /Exercicio1-jsons/

processar_todas_as_paginas()

[SUCESSO] | Informacoes extraidas e salvas                                  => .cd.json
[SUCESSO] | Informacoes extraidas e salvas                                  => .ps.json
[SUCESSO] | Informacoes extraidas e salvas                                  => .tr.json
[ATENCAO] | 1.ª_reunião_de_cúpula_do_G20.html | Nao contem infobox
[ATENCAO] | 10.ª_reunião_de_cúpula_do_G20.html | Nao contem infobox
[ATENCAO] | 11.ª_reunião_de_cúpula_do_G20.html | Nao contem infobox
[ATENCAO] | 12.ª_reunião_de_cúpula_do_G20.html | Nao contem infobox
[ATENCAO] | 13.ª_reunião_de_cúpula_do_G20.html | Nao contem infobox
[ATENCAO] | 14.ª_reunião_de_cúpula_do_G20.html | Nao contem infobox
[ATENCAO] | 15.ª_reunião_de_cúpula_do_G20.html | Nao contem infobox
[ATENCAO] | 1566.html | Nao contem infobox
[ATENCAO] | 15_de_janeiro.html | Nao contem infobox
[ATENCAO] | 15_de_novembro.html | Nao contem infobox
[ATENCAO] | 16.ª_reunião_de_cúpula_do_G20.html | Nao contem infobox
[ATENCAO] | 1625.html | Nao contem infobox
[A